In [1]:
import os
import pandas as pd
import janitor
import numpy as np
pd.set_option('display.max_columns', 100)

DATAPATH = "../adult-data"        

In [2]:
# (fold cell) Load and prep web browsing data
# output = df with visits to adult sites
usecols_web_desktop = ["caseid", "category", "private_domain", "visit_duration", "visit_time_local"]
usecols = ["caseid", "category", "private_domain", "page_duration", "session_start_time"]

df = (pd.concat([
        # Get web_mobile
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_mobile_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ), 
        # Get web_desktop
        pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_desktop_2022-06-01_2022-06-30.csv"),
            usecols=usecols_web_desktop,
            low_memory=False,
        ),
        # Get web
        (pd.read_csv(
            os.path.join(DATAPATH, "output/realityMine_web_2022-06-01_2022-06-30.csv"),
            usecols=usecols,
            low_memory=False,
        )
         # Renaming columns to be consistent w/ web_mobile & web_desktop
         .rename_column("session_start_time", "visit_time_local")
         .rename_column("page_duration", "visit_duration")
        )
    ])
     )
print("output = df with web browses")
print(f"{len(df)=:,}")
df.head()

output = df with web browses
len(df)=6,297,382


,caseid,private_domain,category,visit_time_local,visit_duration
0,205323077,google.com,Search Engines and Portals,2022-05-31 23:52:37,2
1,205323077,coupons.com,"Business, Shopping",2022-06-01 01:07:35,457
2,205323077,google.com,Business,2022-06-01 01:15:12,55
3,205323077,coupons.com,"Business, Shopping",2022-06-01 01:16:07,2225
4,205323077,google.com,Search Engines and Portals,2022-06-01 04:38:10,10


In [3]:
df["private_domain"].unique()

array(['google.com', 'coupons.com', 'myfwc.com', ...,
       'streetsideclassics.com', 'cryptopolitan.com', 'mychart.pr'],
      dtype=object)

In [16]:
df_domains = (
    df
    .dropna(subset=["private_domain"])
    .select_columns(["private_domain", "category"])
    .drop_duplicates(["private_domain", "category"])
    .sort_values("private_domain", ignore_index=True)
)
df_domains.to_csv("domains_categories.csv", index=False)
df_domains

,private_domain,category
0,007james.com,"Business, Entertainment"
1,0123movie.net,"Entertainment, Illegal Content"
2,0410690.com,NaN
3,042jam.com,NaN
4,042nobs.com,NaN
...,...,...
69846,zyvr.com,NaN
69847,zzounds.com,"Business, Shopping"
69848,zztop.com,NaN
69849,zzzperform.com,NaN


In [15]:
df_domains.query("private_domain=='google.com'")

,private_domain,category
24832,google.com,"Business, Education"
24833,google.com,"Business, Chat and Instant Messaging, Informat..."
24834,google.com,Business
24835,google.com,Media Sharing
24836,google.com,News and Media
24837,google.com,"Information Technology, Shopping"
24838,google.com,"Business, Education, Information Technology"
24839,google.com,Translation Sites
24840,google.com,Information Technology
24841,google.com,NaN


In [4]:
df["private_domain"].nunique()

64074